In [1]:
from llm_datasets import LLMDatabaseBuilder
from tune import LLMTuner
from utils import load_yaml
import textwrap

/home/cisanche/miniconda3/envs/llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/cisanche/miniconda3/envs/llm/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/cisanche/miniconda3/envs/llm/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
settings = load_yaml('server.config.yml')

In [3]:
dataset = LLMDatabaseBuilder(settings)
dataset.promt_base

'You are a {model}. {task}: {categories_description} The tweet text is delimited with triple backticks.'

In [4]:
dataframe_sample = {
    'tweet': 'Praying for Haiti. Hoping for better news on Hurricane Matthew the next few days.',
    'task': 'humanitarian', 
    'explanation': 'Tweet expresses hope and prayer for a crisis situation.',
    'label': 'Sympathy and support',
}

sample = dataset.join_dataframe_and_prompt(dataframe_sample)
print(textwrap.fill(sample, width=120))

<s>[INST] <<SYS>> You are a social media message classifier. Your task is to categorize a tweet posted during crisis
events as one of the following humanitarian categories: - Affected individuals: information about deaths, injuries,
missing, trapped, found or displaced people, including personal updates about oneself, family, or others. -
Infrastructure and utilities: information about buildings, roads, utilities/services that are damaged, interrupted,
restored or operational. - Donations and volunteering: information about needs, requests, queries or offers of money,
blood, shelter, supplies (e.g., food, water, clothing, medical supplies) and/or services by volunteers or professionals.
- Caution and advice: information about warnings issued or lifted, guidance and tips. - Sympathy and support: thoughts,
prayers, gratitude, sadness, etc. - Other useful information: information NOT covered by any of the above categories. It
considers information about fire line/emergency location, flood

In [5]:
databases = dataset.get_datasets()

In [6]:
train, valid = databases

In [7]:
settings['train']['batch_size'] = 4
settings['train']['max_steps'] = 64

In [8]:
tuner = LLMTuner(settings)

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.96s/it]
/home/cisanche/miniconda3/envs/llm/lib/python3.8/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [10]:
tuner.train(train, valid)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/cisanche/miniconda3/envs/llm/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


/home/cisanche/miniconda3/envs/llm/lib/python3.8/site-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
